# Use case for the Bridge 2021 fire

In [1]:
from dsfunctions import forward_pass, forward_pass_nsteps, sample_windspeed, sample_winddirection, sample_geometry
from dsfunctions import plot_matrix, plot_matrix_ensemble, align_geoms, interpolate_geom, get_observation, plot_geometry
from dsfunctions import geom_to_vector, reproject_geom, validate_geom
from dsfunctions import forward_pass_farsite

import farsiteutils_v2 as futils
import datetime
import uuid

import numpy as np
from matplotlib import pyplot as plt

import contextily as ctx

import pickle
from shapely.geometry import Polygon
from tqdm import tqdm

description = 'Bridge2021'

# with open(f'./Bridge2021_results_22082024_vsz1_wsz100.pkl', 'rb') as f:
#     data = pickle.load(f)

# with open(f'./Bridge2021_results_22082024_vsz1_wsz1.pkl', 'rb') as f:
#     data = pickle.load(f)

with open(f'./Bridge2021_results_22082024_vsz100_wsz100.pkl', 'rb') as f:
    data = pickle.load(f)

# with open(f'./Bridge2021_results_22082024_vsz100_wsz1.pkl', 'rb') as f:
#     data = pickle.load(f)

# with open(f'./Bridge2021_results_22082024_vsz1000_wsz1.pkl', 'rb') as f:
#     data = pickle.load(f)


adjusted_geom_lst = data['adjusted_geom']
zkphat_ensemble_lst = data['zkphat_ensemble_lst']
xkphat_ensemble_lst = data['xkphat_ensemble_lst']
ykhat_ensemble_lst = data['ykhat_ensemble_lst']

tfin = 6

# # # Bridge fire wind values
wslst = [15]*(tfin)
wdlst = [0]*(tfin)

observations = []
timestamps = []
sampled_wslst = []
sampled_wdlst = []

for tix in range(tfin):
    obs, ts = get_observation(description, tix)
    observations.append(obs)
    timestamps.append(ts)

# # Bridge2021 ignition
ignition_timepoint = timestamps[0].replace(hour=12, minute=53)
x,y = x,y = observations[0].centroid.x, observations[0].centroid.y
width = 100
xshift = -50
yshift = 50

x += xshift
y+= yshift
ignition = Polygon([(x-width, y), (x, y), (x,y-width), (x-width, y-width)])

/opt/conda/lib/python3.11/site-packages/geopandas/_compat.py:124: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


## Timestep 0

In [7]:
for tp in ([ignition_timepoint] + timestamps):
    print(tp)

2021-09-05 12:53:52
2021-09-05 14:04:52
2021-09-05 14:34:29
2021-09-05 15:02:16
2021-09-05 15:25:20
2021-09-05 15:58:27
2021-09-05 16:23:31
